# Plot modulated rate trajectories

Push of the past / pull of the present

In [1]:
import numpy as np
from bdms import poisson
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

import my_bdms
import modulators

In [2]:
with open('plot_settings.yml', 'r') as file:
    config = yaml.safe_load(file)

for key, value in config['paper']['rcParams'].items():
    plt.rcParams[key] = value

legend_info = config['paper']['legend_info']
legend_labels = legend_info['legend_labels']
fig_dpi = config['paper']['fig_dpi']
palette = sns.color_palette(legend_info['sns_palette'], len(legend_info['mode_order']))

We define a 2-state process with the two states having different birth rates, similar death rates, and similar mutation rates.
We take the sampling probability to be $0.1$.

In [3]:
low_rho = .01
high_rho = .25

modulator_low_rho = modulators.FEModulator(
    state_space=np.array([0, 1]),
    birth_rates=np.array([0.25, 1.0]),
    death_rates=np.array([0.25, 0.25]),
    mutation_rates=np.array([0.5, 0.5]),
    transition_matrix=np.array([[0, 1], [1, 0]]),
    rhos=low_rho * np.ones(2),
    t_min=0.0,
    t_max=10.0,
    dt=0.01,
)
FE_birth_process_low_rho = my_bdms.CustomProcess(modulator_low_rho.λ, modulator_low_rho.Λ, modulator_low_rho.Λ_inv)
FE_death_process_low_rho = poisson.ConstantProcess(0.0)
FE_mutation_process_low_rho = my_bdms.CustomProcess(modulator_low_rho.m, modulator_low_rho.M, modulator_low_rho.M_inv)

modulator_high_rho = modulators.FEModulator(
    state_space=np.array([0, 1]),
    birth_rates=np.array([0.25, 1.0]),
    death_rates=np.array([0.25, 0.25]),
    mutation_rates=np.array([0.5, 0.5]),
    transition_matrix=np.array([[0, 1], [1, 0]]),
    rhos=high_rho * np.ones(2),
    t_min=0.0,
    t_max=10.0,
    dt=0.01,
)
FE_birth_process_high_rho = my_bdms.CustomProcess(modulator_high_rho.λ, modulator_high_rho.Λ, modulator_high_rho.Λ_inv)
FE_death_process_high_rho = poisson.ConstantProcess(0.0)
FE_mutation_process_high_rho = my_bdms.CustomProcess(modulator_high_rho.m, modulator_high_rho.M, modulator_high_rho.M_inv)

In [4]:
t = np.arange(modulator_low_rho.t_min, modulator_low_rho.t_max + modulator_low_rho.dt, modulator_low_rho.dt)

Note that we reverse the time direction to align with manuscript notation.

In [5]:
%%capture

fig, axes = plt.subplots(5, 1, figsize=(2.5, 4))

for state, state_label in zip(modulator_low_rho.state_space, ("Unfit", "Fit")):
    axes[0].plot([modulator_low_rho.t_min, modulator_low_rho.t_max],
                 [modulator_low_rho.birth_rates[state]] * 2,
                    color=f"C{state}", 
                    label=f"{state_label}, full")
    FE_birth_rate_trajectory_low_rho = [FE_birth_process_low_rho.λ(state, ti) for ti in t]
    FE_birth_rate_trajectory_high_rho = [FE_birth_process_high_rho.λ(state, ti) for ti in t]
    axes[0].plot(modulator_low_rho.t_max - t, FE_birth_rate_trajectory_low_rho,
                 linestyle = ":",
                 label=rf"{state_label}, FE, $\rho = {low_rho:.2f}$", color=f"C{state}")
    axes[0].plot(modulator_high_rho.t_max - t, FE_birth_rate_trajectory_high_rho,
                 linestyle = "--",
                 label=rf"{state_label}, FE, $\rho = {high_rho:.2f}$", color=f"C{state}")
    axes[2].plot([modulator_low_rho.t_min, modulator_low_rho.t_max],
                 [modulator_low_rho.mutation_rates[state]] * 2,
                    color=f"C{state}",
                    label=f"{state_label}, full")
    FE_mutation_rate_trajectory_low_rho = [FE_mutation_process_low_rho.λ(state, ti) for ti in t]
    FE_mutation_rate_trajectory_high_rho = [FE_mutation_process_high_rho.λ(state, ti) for ti in t]
    axes[2].plot(modulator_low_rho.t_max - t, FE_mutation_rate_trajectory_low_rho,
                 linestyle = ":",
                 label=rf"{state_label}, FE, $\rho = {low_rho:.2f}$", color=f"C{state}")
    axes[2].plot(modulator_high_rho.t_max - t, FE_mutation_rate_trajectory_high_rho,
                 linestyle = "--",
                 label=rf"{state_label}, FE, $\rho = {high_rho:.2f}$", color=f"C{state}")
    survival_trajectory_low_rho = [modulator_low_rho._s(ti, state) for ti in t]
    survival_trajectory_high_rho = [modulator_high_rho._s(ti, state) for ti in t]
    axes[4].plot(modulator_low_rho.t_max - t, survival_trajectory_low_rho,
                    linestyle = ":",
                    label=rf"{state_label}, FE, $\rho = {low_rho:.2f}$", color=f"C{state}")
    axes[4].plot(modulator_high_rho.t_max - t, survival_trajectory_high_rho,
                    linestyle = "--",
                    label=rf"{state_label}, FE, $\rho = {high_rho:.2f}$", color=f"C{state}")

axes[0].set_ylabel(r"birth rate")
axes[0].legend(loc='upper left', bbox_to_anchor=(0, .95))
            #    fontsize="small")
# Legend
handles = axes[0].get_legend().legend_handles
labels = [text.get_text() for text in axes[0].get_legend().get_texts()]
axes[0].get_legend().remove()  # Remove the legend from the current plot, but keep the plot itself

# axes[2].set_xlabel("time before present")
axes[2].set_ylabel(r"mutation rate")

axes[1].axhline(modulator_low_rho.birth_rates[1] / modulator_low_rho.birth_rates[0], color="k", label="full")
FE_relative_birth_rate_trajectory_low_rho = [FE_birth_process_low_rho.λ(1, ti) / FE_birth_process_low_rho.λ(0, ti) for ti in t]
FE_relative_birth_rate_trajectory_high_rho = [FE_birth_process_high_rho.λ(1, ti) / FE_birth_process_high_rho.λ(0, ti) for ti in t]
axes[1].plot(modulator_low_rho.t_max - t, FE_relative_birth_rate_trajectory_low_rho, linestyle=":", label=f"FE", color="k")
axes[1].plot(modulator_low_rho.t_max - t, FE_relative_birth_rate_trajectory_high_rho, linestyle="--", label=f"FE", color="k")
axes[1].set_ylabel("birth rate (rel)")
axes[1].legend(loc='upper right', fontsize="small")
rel_handles = axes[1].get_legend().legend_handles
rel_labels = [text.get_text() for text in axes[1].get_legend().get_texts()]
axes[1].get_legend().remove()  # Remove the legend from the current plot, but keep the plot itself

axes[3].axhline(modulator_low_rho.mutation_rates[1] / modulator_low_rho.mutation_rates[0], color="k")
FE_relative_mutation_rate_trajectory_low_rho = [FE_mutation_process_low_rho.λ(1, ti) / FE_mutation_process_low_rho.λ(0, ti) for ti in t]
FE_relative_mutation_rate_trajectory_high_rho = [FE_mutation_process_high_rho.λ(1, ti) / FE_mutation_process_high_rho.λ(0, ti) for ti in t]
axes[3].plot(modulator_low_rho.t_max - t, FE_relative_mutation_rate_trajectory_low_rho, linestyle=":", color="k")
axes[3].plot(modulator_low_rho.t_max - t, FE_relative_mutation_rate_trajectory_high_rho, linestyle="--", color="k")
axes[3].set_ylabel("mutation rate (rel)")

axes[4].set_ylabel(r'$1-E_a(\tau)$')
axes[4].set_xlabel("time before present")

plt.tight_layout()
plt.savefig("fig/POTP.pdf")
plt.show()

In [6]:
%%capture

columnwise_to_rowwise = [0,3,2,5,1,4]

# Change labels to desired labels
# labels = [legend_info['legend_labels'][label] for label in labels]
# Create a new figure just for the legend
legend_fig, legend_ax = plt.subplots(figsize=(2, .1))  # Create a new subplot
# Add the legend to the subplot
legend_ax.legend([handles[idx] for idx in columnwise_to_rowwise],
                 [labels[idx] for idx in columnwise_to_rowwise],
                 loc='center', ncol = 3)
legend_ax.axis('off')  # Turn off the axis

# Draw the legend figure canvas before saving or showing
legend_fig.canvas.draw()
plt.tight_layout()

# Save the legend to a file
plt.savefig('fig/POTP_legend.pdf', bbox_inches='tight')

# Change labels to desired labels
# labels = [legend_info['legend_labels'][label] for label in labels]
# Create a new figure just for the legend
legend_fig, legend_ax = plt.subplots(figsize=(1, .1))  # Create a new subplot
# Add the legend to the subplot
legend_ax.legend(rel_handles, rel_labels, loc='center', ncol = 2)
legend_ax.axis('off')  # Turn off the axis

# Draw the legend figure canvas before saving or showing
legend_fig.canvas.draw()
plt.tight_layout()

# Save the legend to a file
plt.savefig('fig/POTP_rel_legend.pdf', bbox_inches='tight')